In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [48]:
## Initial Imports

%matplotlib inline

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import LeaveOneOut, GridSearchCV, KFold, train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, matthews_corrcoef, roc_auc_score
from statistics import mean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek
import imblearn
import joblib
from collections import Counter
random.seed(10)

In [49]:
#pdh = pd.read_csv(r'C:\Users\User\Videos\PD paper smote oversampling/gender_class_FH_corrected_Copy.csv')
pdh = pd.read_csv(r'C:\Users\User\Videos\PD paper smote oversampling/tqwt_mfccs_concat_FH.csv')
print(pdh.head())
#print('The shape of the matrix is :', pdh.shape)
print(list(pdh.columns))

   tqwt_maxValue_dec_14  tqwt_entropy_log_dec_33  tqwt_meanValue_dec_17  \
0              0.105750               -4540.0830           9.600000e-20   
1              0.121580               -4466.8949          -1.180000e-19   
2              0.081818               -4455.2826          -7.420000e-20   
3              0.567740               -2779.8086          -1.070000e-19   
4              0.567490               -2910.9996           1.310000e-19   

   tqwt_meanValue_dec_27  tqwt_entropy_log_dec_35  tqwt_entropy_log_dec_10  \
0           1.840000e-18               -3190.1752             -281439.2690   
1          -4.270000e-18               -3106.4317             -285733.2255   
2          -2.120000e-18               -3082.5691             -305145.0185   
3           5.940000e-19               -1965.3808             -287896.7686   
4           1.280000e-19               -2102.3852             -279129.6866   

   tqwt_meanValue_dec_3  tqwt_entropy_log_dec_28  tqwt_meanValue_dec_6  \
0     

In [50]:
df=pdh
train_df=df.values
x_train=train_df[:,:-1]
y_train=train_df[:,-1]
print(Counter(y_train))
print(x_train.shape)
print(y_train.shape)
print(len(y_train))

Counter({1.0: 564, 0.0: 192})
(756, 118)
(756,)
756


In [51]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train= sc.fit_transform(x_train)
#newly added

kfold=KFold(252, shuffle=False)#random_state=10

In [52]:
def oversampling_smote(trainx,trainy, xs=2):
    
    # uses concept of smote for oversampling
    
    attrs=trainx.shape[1] # getting number of attributes
    synthetic_sample=np.empty((3*(xs-1), attrs))
    overx=trainx[0:3,:]
    overy=[]
    
    for i in range(0,len(trainx),3):
        if(trainy[i]==0):
            gap1=trainx[i]-trainx[i+1]
            gap2=trainx[i]-trainx[i+2]
            gap3=trainx[i+1]-trainx[i+2]
            
            for j in range(synthetic_sample.shape[0],3):
                synthetic_sample[j]=trainx[i]+random.uniform(0,1)*gap1#trainx[i]+random.uniform(0,1)*gap1
                synthetic_sample[j+1]=trainx[i+2]+random.uniform(0,1)*gap2
                synthetic_sample[j+2]=train[i+1]+random.uniform(0,1)*gap3
            
            overx=np.vstack((overx,trainx[i:i+3,:]))
            overx=np.vstack((overx, synthetic_sample))
            
            for j in trainy[i:(i+3)]:
                overy.append(j)
            
            for l in range(synthetic_sample.shape[0]):
                overy.append(trainy[i])
            
        else:
            overx=np.vstack((overx,trainx[i:i+3,:]))
            for j in trainy[i:i+3]:
                overy.append(j)
    overx=overx[3:,:]#cropping of the 1st 3 because it is reapeated
    overy=np.array(overy)
    print(overx.shape)
    print(overy.shape)
    print(Counter(overy))
    return overx,overy

In [53]:
wrong_pred_indices=[]
d={}
svm_estimator=svm.SVC(C= 12, gamma= 0.001, kernel= 'poly',probability=True, random_state=10)
i=0

correct_ones_=[]#collects correct pd postive predictions
correct_zeros_=[]#collects correct pd negative predictions

wrong_ones_=[]#collect wrong pd positive predictions
wrong_zeros_=[]#collect wrong pd negative predictions

tp=[]# true positive
tn=[]# true negative
fp=[]# false positive
fn=[]# false negative

tridx=[]
tstidx=[]
acc=[]
mcc=[]
f1=[]
tsts=[]

for trids, tstids in kfold.split(x_train):
    
    trainx, testx= x_train[trids,:], x_train[tstids,:]
    trainy, testy= y_train[trids], y_train[tstids]
    

    trainx, trainy=oversampling_smote(trainx,trainy, xs=2)
    over_samp=SMOTETomek(sampling_strategy='not majority')
    trainx, trainy=over_samp.fit_resample(trainx, trainy)
    
    svm_estimator=svm.SVC(C= 10, gamma= 0.001, tol=1e-3, kernel= 'rbf',probability=True, random_state=10)
    
    svm_estimator.fit(trainx, trainy)
    
    preds=svm_estimator.predict(x_train[tstids,:])
    
    joblib.dump(svm_estimator,r'C:\Users\User\Videos\PD paper smote oversampling\files for oversampling\Saved Models-'+str(i))
    
    acc.append(accuracy_score(testy,svm_estimator.predict(testx)))
    
    #f1.append(f1_score(testy, svm_estimator.predict(testx)))
    

    i=i+1
    print(i)
print('raw acc:', mean(acc))
#print('f1:', mean(f1))
#print('mcc:', mean(mcc))


(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
1
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
2
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
3
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
4
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
5
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
6
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
7
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
8
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
9
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
10
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
11
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
12
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
13
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
14
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
15
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
16
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
17
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
18
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
19
(945, 118)
(945,)
Counter({1.0: 561, 0.0

160
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
161
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
162
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
163
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
164
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
165
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
166
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
167
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
168
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
169
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
170
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
171
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
172
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
173
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
174
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
175
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
176
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
177
(945, 118)
(945,)
Counter({1.0: 561, 0.0: 384})
178
(942, 118)
(942,)
Counter({1.0: 564, 0.0: 378})
179
(945, 11

In [54]:
j=0
tpids=[]
tpc=0
tnids=[]
tnc=0
fpids=[]
fpc=0
fnids=[]
fnc=0

#performing majority voting
# if accurately predicted over 0.66 for 3 that means we predicted correctly for the whole sample

for t in range(0,len(y_train),3):
    print('t:',t," original:", y_train[t],' patient no: ',j, 'correct prediction rate:',acc[j] )

    
    if y_train[t]==1 and acc[j]>0.5:
        tpc=tpc+1
        tpids.append(t)
    if y_train[t]==1 and acc[j]<0.5:
        fnc=fnc+1
        fnids.append(t)
    if y_train[t]==0 and acc[j]>0.5:
        tnc=tnc+1
        tnids.append(t)
    if y_train[t]==0 and acc[j]<0.5:
        fpc=fpc+1
        fpids.append(t)
    
    j=j+1

print("true positives: ", tpc)
print("true negatives: ", tnc)
print("fales positives: ", fpc)
print("false negatives: ", fnc)

t: 0  original: 1.0  patient no:  0 correct prediction rate: 0.0
t: 3  original: 1.0  patient no:  1 correct prediction rate: 1.0
t: 6  original: 1.0  patient no:  2 correct prediction rate: 0.3333333333333333
t: 9  original: 1.0  patient no:  3 correct prediction rate: 1.0
t: 12  original: 1.0  patient no:  4 correct prediction rate: 1.0
t: 15  original: 1.0  patient no:  5 correct prediction rate: 1.0
t: 18  original: 1.0  patient no:  6 correct prediction rate: 1.0
t: 21  original: 1.0  patient no:  7 correct prediction rate: 1.0
t: 24  original: 1.0  patient no:  8 correct prediction rate: 0.6666666666666666
t: 27  original: 1.0  patient no:  9 correct prediction rate: 1.0
t: 30  original: 1.0  patient no:  10 correct prediction rate: 1.0
t: 33  original: 0.0  patient no:  11 correct prediction rate: 0.0
t: 36  original: 1.0  patient no:  12 correct prediction rate: 1.0
t: 39  original: 1.0  patient no:  13 correct prediction rate: 1.0
t: 42  original: 0.0  patient no:  14 correct 

t: 477  original: 1.0  patient no:  159 correct prediction rate: 1.0
t: 480  original: 1.0  patient no:  160 correct prediction rate: 1.0
t: 483  original: 1.0  patient no:  161 correct prediction rate: 1.0
t: 486  original: 1.0  patient no:  162 correct prediction rate: 1.0
t: 489  original: 0.0  patient no:  163 correct prediction rate: 0.0
t: 492  original: 1.0  patient no:  164 correct prediction rate: 1.0
t: 495  original: 1.0  patient no:  165 correct prediction rate: 1.0
t: 498  original: 1.0  patient no:  166 correct prediction rate: 1.0
t: 501  original: 1.0  patient no:  167 correct prediction rate: 1.0
t: 504  original: 1.0  patient no:  168 correct prediction rate: 1.0
t: 507  original: 1.0  patient no:  169 correct prediction rate: 1.0
t: 510  original: 1.0  patient no:  170 correct prediction rate: 1.0
t: 513  original: 1.0  patient no:  171 correct prediction rate: 1.0
t: 516  original: 1.0  patient no:  172 correct prediction rate: 0.0
t: 519  original: 0.0  patient no:

In [55]:
f1=0
f1=(tpc)/((tpc+0.5*(fpc+fnc)))
print('f1 score:', f1)

f1 score: 0.917098445595855


In [56]:
import math
mcc=0

mcc=(tpc*tnc-fpc*fnc)/math.sqrt((tpc+fpc)*(tpc+fnc)*(tnc+fpc)*(tnc+fnc))
print('mcc: ',mcc)
precision=tpc/(tpc+fpc)
print('precision: ', precision)
recall=tpc/(tpc+fnc)
print('recall: ',recall)
acc=(tpc+tnc)/(tpc+tnc+fpc+fnc)
print('accuracy: ', acc)

mcc:  0.6506657925275016
precision:  0.8939393939393939
recall:  0.9414893617021277
accuracy:  0.873015873015873


In [58]:
tpc,tnc, fpc, fnc

(177, 43, 21, 11)

In [23]:
import lime
import shap

In [24]:
#would have to do lime and shape later

In [25]:
print(Counter(trainy))

Counter({1.0: 564, 0.0: 564})


In [26]:
np.random.get_state()

('MT19937',
 array([4221090049,  399551028, 1963481888,  924475493, 3240044603,
        2665764951,  318030231, 4280725112, 1191103372, 3698245382,
        3476334861, 3780123126, 2142526952, 1825036343, 2101435196,
         858212940, 2780067505, 2089405991, 3925595345,  882322219,
        2959903781, 3521600608,  594565318, 1679110574, 1366650126,
        3229498570, 2210473618, 2039841540, 1103022237, 2155374657,
         839382938, 3737000349, 1431214791, 3795057993, 2873962084,
        2254900914, 3210727593, 3077412226, 1856503286, 2544252331,
        1337417950, 2711091903,  511282365,  922739507, 1306063600,
        2996758435, 1089937995, 2235488905, 3467611881, 2823772360,
        4043365651, 1626586141, 1878409452, 3874752545, 2499038052,
         806206528,  512637255,  102663133,  401547541, 2114029524,
        3206320696, 3568049511, 3881506690, 3392246456, 2251161582,
         868278724, 1529826228, 2448359147,  338166849, 2417638803,
        3137232539, 1172814840,  247

In [27]:
random.getstate()

(3,
 (2147483648,
  3187044887,
  2687139204,
  3012076618,
  1082909080,
  2165184475,
  159641056,
  1896720589,
  1234731307,
  3884663462,
  2625499442,
  3220313046,
  3360593574,
  438945526,
  2524995985,
  739488192,
  1047842947,
  3379107547,
  1597925988,
  42576134,
  585448826,
  2204387539,
  4179227443,
  2294212805,
  1081134693,
  146644662,
  1276343436,
  2214519178,
  3146211407,
  3485878849,
  294217766,
  577256189,
  1152037460,
  3700418114,
  1883879286,
  297672995,
  3726501623,
  4167353500,
  4021192333,
  434216298,
  1744336951,
  2021645364,
  70671851,
  2020330654,
  2577311338,
  2747036466,
  731618384,
  3874740903,
  44310958,
  2536544235,
  1673857689,
  2880322085,
  402044473,
  3413383964,
  1119938996,
  800033731,
  2034775919,
  2132199888,
  3813699922,
  1793028474,
  4143048811,
  1578969269,
  273902685,
  3561078042,
  3881931017,
  2759718296,
  181963981,
  1781838550,
  1760853640,
  1385615315,
  4286074154,
  1198546185,
  222507

In [59]:
mod=joblib.load('Saved Models-1')

In [61]:
mod.predict(testx)

array([0., 0., 0.])